In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! unzip /content/multilingual-customer-support-tickets.zip

Archive:  /content/multilingual-customer-support-tickets.zip
replace helpdesk_customer_tickets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: helpdesk_customer_tickets.csv  


In [ ]:
! kaggle datasets download tobiasbueck/multilingual-customer-support-tickets

Dataset URL: https://www.kaggle.com/datasets/tobiasbueck/multilingual-customer-support-tickets
License(s): Attribution 4.0 International (CC BY 4.0)
multilingual-customer-support-tickets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import pandas as pd
from transformers import pipeline
import torch
from tqdm import tqdm
tqdm.pandas()
df = pd.read_csv("/content/helpdesk_customer_tickets.csv")

In [ ]:
df.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,36,Anfrage zu den Spezifikationen und Anpassungso...,Sehr geehrtes Support-Team des Tech Online Sto...,"Sehr geehrter <name>,\n\nvielen Dank für Ihr I...",Request,Customer Service,medium,de,Tech Online Store,Product Support,Sales Inquiry,Technical Guidance,General Inquiry,NaN,NaN,NaN,NaN,NaN
1,39,Déconnexions fréquentes et plantages,Le client signale des déconnexions fréquentes ...,Nous allons enquêter sur le problème avec Zoom...,Incident,Product Support,high,fr,Software Development Company,Technical Support,Software Bug,Service Disruption,System Crash,Problem Resolution,Performance Tuning,NaN,NaN,NaN
2,243,Problema de sonido Dell XPS,"Problema con el sonido, manejando como devoluc...",Gracias por su correo electrónico. Procesaremo...,Problem,Returns and Exchanges,medium,es,Tech Online Store,Returns and Exchanges,Product Support,Customer Service,Refund Request,NaN,NaN,NaN,NaN,NaN
3,381,Assistance requise pour la configuration du ta...,"Cher support client,\n\nNotre client, <name>, ...","Cher <name>,\n\nMerci de nous avoir contactés....",Request,Product Support,medium,fr,Software Development Company,Technical Support,Product Support,General Inquiry,Problem Resolution,Training Request,NaN,NaN,NaN,NaN
4,663,Urgente: Assistência Imediata Necessária para ...,Caro Suporte ao Cliente da Firma de Consultori...,"Caro Cliente,\n\nRecebemos sua solicitação urg...",Incident,Human Resources,medium,pt,IT Consulting Firm,Urgent Issue,Payroll Issue,Technical Support,Service Disruption,Problem Resolution,Account Assistance,NaN,NaN,NaN


In [ ]:
df.shape

(600, 18)

In [ ]:
df["queue"].value_counts()

,count
queue,
Technical Support,210
Product Support,93
Customer Service,85
IT Support,77
Billing and Payments,46
Returns and Exchanges,41
Human Resources,15
Service Outages and Maintenance,15
Sales and Pre-Sales,13


In [ ]:
df["language"].value_counts()

,count
language,
en,163
de,152
es,133
pt,81
fr,71


In [ ]:
df["content"] = df["subject"] + ' ' + df["body"]
df_emails = df[["content","language","queue"]]
df_emails.isnull().sum()

,0
content,1
language,0
queue,0


In [ ]:
df_emails = df_emails.loc[df['language'] != 'pt']


df_emails["language"].value_counts()

In [ ]:
df.dropna(inplace = True)

In [ ]:
device = 0 if torch.cuda.is_available() else -1
translator_de = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en", device = device)
translator_es = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en", device = device)
translator_fr = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device = device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default.

In [ ]:
def get_translator(language):
    if language == 'es':
        return translator_es
    elif language == 'fr':
        return translator_fr
    elif language == 'de':
        return translator_de
    elif language == 'en':
        return None

# Function to translate text
def translate_text(row):
    translator = get_translator(row['language'])
    if translator is not None:
        translation = translator(row['content'], batch_size=4)
        return translation[0]['translation_text']
    return row['content']  # If no translator, return original text

# Sample DataFrame (for demonstration)
data = {
    'content': ['Hola, ¿cómo estás?', 'Bonjour tout le monde!', 'Hallo, wie geht es dir?', 'This is already English'],
    'language': ['es', 'fr', 'de', 'en']
}
df_test = pd.DataFrame(data)

# Apply the translation function to each row in the DataFrame
df_test['translated_text'] = df_test.apply(translate_text, axis=1)
df_test

,content,language,translated_text
0,"Hola, ¿cómo estás?",es,"Hey, how are you?"
1,Bonjour tout le monde!,fr,"Hello, everybody!"
2,"Hallo, wie geht es dir?",de,"Hello, how are you?"
3,This is already English,en,This is already English


In [ ]:
tqdm.pandas()
df_emails['translated_text'] = df_emails.apply(translate_text, axis=1)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df_emails

,content,language,queue,translated_text
0,Anfrage zu den Spezifikationen und Anpassungso...,de,Customer Service,Request for specifications and customization o...
1,Déconnexions fréquentes et plantages Le client...,fr,Product Support,Frequent Logouts and crashes The customer repo...
2,Problema de sonido Dell XPS Problema con el so...,es,Returns and Exchanges,"Dell XPS Sound Problem Sound Problem, handling..."
3,Assistance requise pour la configuration du ta...,fr,Product Support,Support required for the configuration of the ...
6,NaN,en,Returns and Exchanges,NaN
...,...,...,...,...
594,Échange demandé pour un ordinateur portable dé...,fr,Returns and Exchanges,Exchange requested for a faulty laptop Dear Cu...
595,Urgente: Problema de Conectividad del Router E...,es,IT Support,Urgent: Router Connectivity Problem Dear IT Se...
596,Necesito ayuda con el despliegue de un nuevo m...,es,Technical Support,I need help with deploying a new microservice ...
597,Necesito soporte urgente Estimado soporte al c...,es,Technical Support,"I need urgent support Dear customer support, T..."


In [ ]:
df_emails.to_csv("translated_emails.csv", index = False)

In [ ]:
df_emails.isnull().sum()

,0
content,1
language,0
queue,0
translated_text,1
